# Database test

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
DB_NAME = "jobFinder"
COLLECTION_NAME = "cv_mehdi"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_search_index"
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGO_URI")

In [3]:
mongo_client = MongoClient(MONGODB_ATLAS_CLUSTER_URI, server_api=ServerApi('1'))
collection = mongo_client[DB_NAME][COLLECTION_NAME]

In [4]:
embeddings = OpenAIEmbeddings()#disallowed_special=())
vector_search = MongoDBAtlasVectorSearch(collection, embedding=embeddings, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME, embedding_key="embedding")

In [5]:
try:
    mongo_client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [6]:
query = "certifications"

docs = vector_search.similarity_search_with_score(query, k=3)
for doc in docs:
    print("score :", doc[1], " ",doc[0].page_content)
    print("-----------------------------------------------------")

score : 0.8940367102622986   
- French TCF C2
- English TOIEC C1*
- Arabic (native language)
- German A1

-----------------------------------------------------
score : 0.8802566528320312   
Programming Languages:
- Python: Proficient for 6 years, used in data analysis, machine learning, and web development.
- C++ and C: Proficient for 6 years, applied in software development and algorithm design.
- Matlab: Experience of 4 years, primarily used for scientific computing, numerical analysis, and algorithm prototyping.
- JavaScript: Proficiency of 3 years, employed in web development for frontend interactivity and functionality.
- SQL for database management (SQLite, MySQL, PostgreSQL)
Frameworks:
- Frontend: Proficient in React and Svelte, used for developing dynamic and responsive user interfaces.
- Backend: Proficient in Flask, Streamlit, and FastAPI, employed for creating backend services, APIs, and web applications.
- Machine Learning: Competent in TensorFlow and PyTorch, utilized for

In [7]:
prompt_template ="""
    You are a smart agent that can answer questions about CVs.
    if the question is not related to CVs, you should answer "I don't know please refer to Mehdi for more information".
    if the question is related to CVs, you should answer the question.
    If the question was not clear enough, you should ask for more information.
    if the question was in French, you should answer in French.
    use the following information to answer the question:
    {context}
    question: {question}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
    return_vector=False,
)
llm = OpenAI(temperature=0.3, max_tokens=200, top_p=0.9)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=qa_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

docs = qa({"query": "gpt-4 compute requirements"})

print(docs["result"])


I don't know please refer to Mehdi for more information.


In [10]:
docs = qa({"query": "how old is Mehdi?"})
print(docs["result"])

Answer: Mehdi is 21 years old, born on January 1st, 2000.


# Agent section

In [11]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import tool
from datetime import datetime

In [16]:
@tool("current_date", return_direct=False)
def current_date(input:str) -> str:
  """Used to know the current date"""
  return str(datetime.now())

@tool("mehdi_cv_data", return_direct=False)
def mehdi_cv_data(input:str) -> str:
  """Used as bank of data for Mehdi CV the input should be a question related to the task to perform"""
  context_helper = "\n".join([ x[0].page_content for x in vector_search.similarity_search_with_score(input, k=5)])
  return context_helper

@tool("meaning_search", return_direct=False)
def meaning_search(input:str) -> str:
  """Used when you look for the meaning of a word"""
  llm = OpenAI(temperature=0.3, max_tokens=50, top_p=0.9)
  return llm(f"What is {input}?")

@tool("no_answer_found", return_direct=True)
def no_answer_found(input:str) -> str:
  """Used when the question provided by the user was not found in the data bank example : when did Mehdi started working at Google? => no answer found"""
  llm = OpenAI(temperature=0.8, max_tokens=50, top_p=0.9)
  return str(llm(f"answer with something that has the following meaning: I don't know or the input provided does not match any of my database information please refer to Mehdi for more information"))
  

@tool("out_of_context", return_direct=True)
def out_of_context(input:str) -> str:
  """Used when the question provided by the user is not related to the task to perform example : what is the weather today?"""
  message = f"I don't know please refer to Mehdi for more information"
  
  return message


In [17]:
llm = ChatOpenAI(temperature=0.5, model='gpt-4')
tools = load_tools(["llm-math"], llm=llm)
tools.append(current_date)
tools.append(mehdi_cv_data)
tools.append(meaning_search)
tools.append(no_answer_found)
tools.append(out_of_context)

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [19]:
agent.run("Give me his linkedin url?")



> Entering new AgentExecutor chain...
I need to find Mehdi's LinkedIn URL from his CV data.
Action: mehdi_cv_data
Action Input: What is Mehdi's LinkedIn URL?
Observation:  - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.


- 2nd prize

"Mehdi's LinkedIn URL is https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/."

In [20]:
agent.run("How old is Mehdi?")



> Entering new AgentExecutor chain...
I need to check Mehdi's birthdate in his CV data to calculate his age.
Action: mehdi_cv_data
Action Input: "What is Mehdi's birthdate?"
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161b

'Mehdi is 23 years old.'

In [21]:
agent.run("Did Mehdi work at Google?")



> Entering new AgentExecutor chain...
I need to check Mehdi's CV data to see if he has worked at Google.
Action: mehdi_cv_data
Action Input: Did Mehdi work at Google?
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


1.

'No, Mehdi did not work at Google.'

In [22]:
agent.run("How many certifications does Mehdi have? and what are they?")



> Entering new AgentExecutor chain...
The question is asking about Mehdi's certifications, which is information that would be found in his CV. I should use the mehdi_cv_data function to get this information.
Action: mehdi_cv_data
Action Input: How many certifications does Mehdi have? and what are they?
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: 

"\n\nI'm sorry, I don't have that information, please contact Mehdi for more details."

In [23]:
agent.run("wich one is the oldest internship?")



> Entering new AgentExecutor chain...
I need to look at Mehdi's CV data to find out which internship is the oldest.
Action: mehdi_cv_data
Action Input: "What is Mehdi's oldest internship?"
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mec

'The oldest internship is the one at Groupe Renault which occurred between August 2021 and October 2021.'

In [24]:
agent.run("what position did mehdi occupie at GE grid solutions?")



> Entering new AgentExecutor chain...
The question is asking for specific information from Mehdi's CV.
Action: mehdi_cv_data
Action Input: What position did Mehdi occupy at GE Grid Solutions?
Observation: 
1. Full Stack Developer at GE Grid Solutions - Type: `Internship` between [June - September 2023], Role: [Intern], Title: [Web Developer]. Tasks and Achievements: Digitized the quality control process by creating an ETL system. Automated the data pipeline using RPA. Standardized the database (SQL) and developed a modular user interface. Created a dashboard for Failure Mode and Effects Analysis (DFMEA) using Power BI.
2. Data Scientist at AI-Inside - Type: `Internship` between [August 2022 - October 2022], Role: [Intern], Title: [Data Scientist]. Tasks and Achievements: Developed and deployed an application using AI and data scraping to predict car sales prices and identify the most advantageous offers (Flask, MongoDB, TensorFlow, Selenium, Pandas).
3. Industrial Engineer at Groupe 

'Mehdi occupied the position of Full Stack Developer at GE Grid Solutions.'

In [25]:
agent.run("What are the inernships that mehdi did?")



> Entering new AgentExecutor chain...
I need to access Mehdi's CV data to answer this question.
Action: mehdi_cv_data
Action Input: What internships did Mehdi do?
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


1. Ful

'Mehdi has done the following internships:\n1. Full Stack Developer at GE Grid Solutions between June and September 2023. During this internship, he digitized the quality control process by creating an ETL system, automated the data pipeline using RPA, standardized the database (SQL), developed a modular user interface, and created a dashboard for Failure Mode and Effects Analysis (DFMEA) using Power BI.\n2. Data Scientist at AI-Inside between August and October 2022. He developed and deployed an application using AI and data scraping to predict car sales prices and identify the most advantageous offers (Flask, MongoDB, TensorFlow, Selenium, Pandas) during this internship.\n3. Industrial Engineer at Groupe Renault between August and October 2021. He actively collaborated on a production line extension project and conducted in-depth studies on various lifting means during this internship.'

In [27]:
output = agent.run("How can I contact Mehdi?")



> Entering new AgentExecutor chain...
I need to look up Mehdi's contact information.
Action: mehdi_cv_data
Action Input: What is Mehdi's contact information?
Observation:  - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.


1. Arts et 

In [28]:
print(output)

You can contact Mehdi through the following methods: 
- Phone: +33 7 52 07 57 72 
- Email: mehdi.hayani_mechkouri@ensam.eu 
- LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


In [29]:
langues = agent.run("Combien de langues parle Mehdi ?")
print(langues)



> Entering new AgentExecutor chain...
La question concerne les informations sur le CV de Mehdi. Je devrais utiliser l'outil mehdi_cv_data pour obtenir cette information.
Action: mehdi_cv_data
Action Input: Combien de langues parle Mehdi ?
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - Link

In [30]:
output = agent.run("es-ce que Mehdi a une experience en data science?")
print(output)



> Entering new AgentExecutor chain...
The question is asking if Mehdi has experience in data science. This is a question about Mehdi's CV.
Action: mehdi_cv_data
Action Input: "Does Mehdi have experience in data science?"
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.

In [31]:
output = agent.run("How many internships did Mehdi do?")
print(output)



> Entering new AgentExecutor chain...
I need to retrieve this information from Mehdi's CV.
Action: mehdi_cv_data
Action Input: How many internships did Mehdi do?
Observation: 
Hello, I am Mehdi HAYANI MECHKOURI, born on January 1st, 2000. I am of Moroccan nationality. Currently in Paris, France. I am currently pursuing a dual degree in Arts et Métiers ParisTech specializing in Digital Engineering, Data Science, and Artificial Intelligence. I am looking for a 6-month internship starting from February 2023 in the field of AI and data science. I am a curious person with a strong thirst for knowledge. I am someone who seeks to solve problems, especially those related to climate change, food security, and energy security. But I am ready to face any challenge. My preferred sector is programming, and I am fascinated by the potential of AI.

 - Phone: +33 7 52 07 57 72 - Email: mehdi.hayani_mechkouri@ensam.eu - LinkedIn: https://www.linkedin.com/in/mehdi-hayani-mechkouri-9ba3161bb/


1. Full

## Conclusion

The agent gives good answers overall but it needs to use chat gpt 4 in order to give accurate answers